In [8]:
from transformers import pipeline
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_openai import OpenAIEmbeddings #for converting chunks into embeddings
from langchain_chroma import Chroma #database for stroring the embeddings
from langchain_huggingface import HuggingFacePipeline

from langchain_community.document_loaders import ImageCaptionLoader

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.retrievers import EnsembleRetriever
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.chains import LLMChain
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from openai import OpenAI 

import pandas as pd
import tempfile
import shutil
import os
# Dictionary to track temporary directories

In [9]:
load_dotenv()

True

In [10]:
def generate_picture_prompt():
    llm = OpenAI(temperature=0.9)

    # Sample description given by previous llm chain
    description = "The red sweater appears to be in a classic style, with a vibrant hue suggesting it is relatively new or well-maintained. Its clean lines and lack of visible wear make it a versatile piece suitable for both casual and slightly dressier occasions."

    system_prompt = (
            "Generate a detailed prompt to generate an image based on the following description: "
            "You are an experienced stylist. "
            "Make sure the image is full body and has all the components of an outfit (i.e. pants, shirt, shoes, accessories). "
            "Make sure it is for a casual context. "
            "\n\n"
            "{image_desc}"
        )

    prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )

    chain = prompt | llm  | StrOutputParser()

    out_message = chain.invoke({
            "image_desc" : description,
            "input": "Provide a full body outfit inspiration"
        })

    return out_message

In [11]:
client = OpenAI()

response = client.images.generate(
    model="dall-e-4",
    prompt="generate_picture_prompt()",
    size="1024x1024",
    quality="standard",
    n=1,
  )

image_url = response.data[0].url
print(image_url)


BadRequestError: Error code: 400 - {'error': {'code': None, 'message': 'Invalid model dall-e-4. The model argument should be left blank.', 'param': None, 'type': 'invalid_request_error'}}